In [1]:
import os

In [2]:
import subprocess

In [3]:
melt_in_str = """\
# 3d Lennard-Jones melt

units		lj
atom_style	atomic

lattice		fcc 0.8442
region		box block 0 10 0 10 0 10
create_box	1 box
create_atoms	1 box
mass		1 1.0

velocity	all create 3.0 87287

pair_style	lj/cut 2.5
pair_coeff	1 1 1.0 1.0 2.5

neighbor	0.3 bin
neigh_modify	every 20 delay 0 check no

fix		1 all nve

#dump		id all atom 50 dump.melt

#dump		2 all image 25 image.*.jpg type type &
#		axes yes 0.8 0.02 view 60 -30
#dump_modify	2 pad 3

#dump		3 all movie 25 movie.mpg type type &
#		axes yes 0.8 0.02 view 60 -30
#dump_modify	3 pad 3

thermo		50
run		250"""

In [4]:
melt_py_str = """\
from mpi4py import MPI
from lammps import PyLammps

L = PyLammps()
L.file('in.melt')


if MPI.COMM_WORLD.rank == 0:
    pe = L.eval("pe")
    print("Potential Energy:", pe)"""

In [5]:
if not os.path.exists('in.melt'):
    with open('in.melt', 'w') as f:
        f.writelines(melt_in_str)

In [6]:
if not os.path.exists('melt.py'):
    with open('melt.py', 'w') as f:
        f.writelines(melt_py_str)

In [7]:
subprocess.check_output('mpiexec -n 4 python melt.py', shell=True, universal_newlines=True ).split('\n')

['LAMMPS (7 Aug 2019)',
 'OMP_NUM_THREADS environment is not set. Defaulting to 1 thread. (src/comm.cpp:93)',
 '  using 1 OpenMP thread(s) per MPI task',
 'Lattice spacing in x,y,z = 1.6796 1.6796 1.6796',
 'Created orthogonal box = (0 0 0) to (16.796 16.796 16.796)',
 '  1 by 2 by 2 MPI processor grid',
 'Created 4000 atoms',
 '  create_atoms CPU = 0.000712872 secs',
 'Neighbor list info ...',
 '  update every 20 steps, delay 0 steps, check no',
 '  max neighbors/atom: 2000, page size: 100000',
 '  master list distance cutoff = 2.8',
 '  ghost atom cutoff = 2.8',
 '  binsize = 1.4, bins = 12 12 12',
 '  1 neighbor lists, perpetual/occasional/extra = 1 0 0',
 '  (1) pair lj/cut, perpetual',
 '      attributes: half, newton on',
 '      pair build: half/bin/atomonly/newton',
 '      stencil: half/bin/3d/newton',
 '      bin: standard',
 'Setting up Verlet run ...',
 '  Unit style    : lj',
 '  Current step  : 0',
 '  Time step     : 0.005',
 'Per MPI rank memory allocation (min/avg/max)